In [1]:
import numpy as np
import pandas as pd
import os
import datetime as dt
from datetime import datetime
from dotenv import load_dotenv
from clickhouse_driver import Client
from data_wrapper import db_wrapper

In [24]:
load_dotenv()

HOST = os.getenv('HOST')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')
CA = os.getenv('CA')
READONLY = os.getenv('READONLY')
ANALYTICAL_HOST = os.getenv('ANALYTICAL_HOST')
ANALYTICAL_PASSWORD = os.getenv('ANALYTICAL_PASSWORD')

# Подключаемся к боевой базе данных
wrap = db_wrapper.ClickHouseWrapper(host=HOST, user=USERNAME, password=PASSWORD, ca=CA)

# Подключаемся к аналитической БД с правами загрузки
client = Client(host=ANALYTICAL_HOST, user=USERNAME, password=ANALYTICAL_PASSWORD, ca_certs=CA, secure=True)

In [25]:
sensors = [
# Потребление ввода холодильной машины
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total AP energy", 
    "channel": 1, "phase": 0, "name_in_df": "cm_consumption", "mode": "max-min"
},
# Мгновенная мощность компрессоров
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total P", 
    "channel": 2, "phase": 0, "name_in_df": "state1", "mode": "max-min"
},
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total P",
    "channel": 3, "phase": 0, "name_in_df": "state2", "mode": "max-min"
},
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total P",
    "channel": 4, "phase": 0, "name_in_df": "state3", "mode": "max-min"
},
{
    "db": "genesis_arena", "name": "map12e_145", "measurement": "Total P",
    "channel": 1, "phase": 0, "name_in_df": "state4", "mode": "max-min"
},
# Мгновенная мощность конденсаторов
{
    "db": "genesis_arena", "name": "map12e_23", "measurement": "Total P", 
    "channel": 1, "phase": 0, "name_in_df": "condensator4", "mode": "mean"
},
{
    "db": "genesis_arena", "name": "map12e_49", "measurement": "Total P", 
    "channel": 2, "phase": 0, "name_in_df": "condensator1", "mode": "mean"
},
{
    "db": "genesis_arena", "name": "map12e_49", "measurement": "Total P", 
    "channel": 3, "phase": 0, "name_in_df": "condensator2", "mode": "mean"
},
{
    "db": "genesis_arena", "name": "map12e_49", "measurement": "Total P", 
    "channel": 4, "phase": 0, "name_in_df": "condensator3", "mode": "mean"
},
# Мощность циркуляционного насоса
{
    "db": "genesis_arena", "name": "map12e_145", "measurement": "Total P", 
    "channel": 3, "phase": 0, "name_in_df": "power_pump", "mode": "mean"
},
# Температура поверхности льда
{
    "db": "genesis_arena", "name": "msw-v3_2", "measurement": "Temperature", 
    "channel": 0, "phase": 0, "name_in_df": "temp_ice", "mode": "mean"
},
# Внешняя температура
{
    "db": "genesis_arena", "name": "weather_owm", "measurement": "Temperature",
    "channel": 0, "phase": 0, "name_in_df": "temp_outside", "mode": "mean"
},
# Внешняя влажность
{
    "db": "genesis_arena", "name": "weather_owm", "measurement": "Humidity",
    "channel": 0, "phase": 0, "name_in_df": "hum_outside", "mode": "mean"
},
# Внутреняя температура 
{
    "db": "genesis_arena", "name": "msw-v3_175", "measurement": "Temperature",
    "channel": 0, "phase": 0, "name_in_df": "temp_inside", "mode": "mean"
},
# Внутреняя влажность 
{
    "db": "genesis_arena", "name": "msw-v3_175", "measurement": "Humidity",
    "channel": 0, "phase": 0, "name_in_df": "hum_inside", "mode": "mean"
}
]

In [68]:
client.execute('USE akarmanov_test_db')
start = client.execute('SELECT MAX(time) FROM genesis_arena_prediction_ice')[0][0]

start = start - dt.timedelta(hours=3, minutes=10)
end = dt.datetime.now().replace(microsecond=0, second=0) - dt.timedelta(hours=3)
presample = dt.timedelta(minutes=1)

df = wrap.get_particular_sensors(start, end, sensors, presample_time=presample,  without_confidence=True)

In [37]:
df = (df
 .tz_localize(None)
 .reset_index(names=['time'])
 .assign(temp_ice=lambda df: np.where(df['temp_ice'].between(-6, 3), df['temp_ice'], np.NaN))
 .pipe(lambda df: df.ffill().bfill())
 .assign(power_compressors=lambda df: df[['state1', 'state2', 'state3', 'state4']].sum(axis=1))
 .assign(power_condensators=lambda df: df[['condensator1', 'condensator2', 'condensator3', 'condensator4']].sum(axis=1))
 .assign(prediction_temp_ice=lambda df: np.NaN)
)

In [40]:
client.execute('USE akarmanov_test_db')
client.insert_dataframe('''
INSERT INTO "genesis_arena_prediction_ice" (
    time, temp_ice, power_compressors, power_condensators, power_pump,
    cm_consumption, temp_outside, temp_inside, hum_outside, hum_inside,
    prediction_temp_ice
) VALUES
''',
    df[['time', 'temp_ice', 'power_compressors', 'power_condensators', 'power_pump',
    'cm_consumption', 'temp_outside', 'temp_inside', 'hum_outside', 'hum_inside',
    'prediction_temp_ice']],
    settings=dict(use_numpy=True)
)

24

In [71]:
client.query_dataframe('select * from genesis_arena_prediction_ice where time > today()')

,time,temp_ice,power_compressors,power_condensators,power_pump,cm_consumption,temp_outside,temp_inside,hum_outside,hum_inside,prediction_temp_ice
0,2024-03-19 14:58:00,-1.877930,31778.679688,5173.854980,4181.060059,0.591667,2.01,9.791667,86.0,47.075001,NaN
1,2024-03-19 14:59:00,-1.820312,32020.904297,4112.050293,4173.754883,0.945833,2.01,9.791667,86.0,47.375000,NaN
2,2024-03-19 15:00:00,-1.854492,31802.689453,6254.570312,4166.169922,0.929167,2.01,9.758333,86.0,47.474998,NaN
3,2024-03-19 15:01:00,-1.920898,31760.525391,5186.287598,4184.310059,0.875000,2.01,9.687500,86.0,47.450001,NaN
4,2024-03-19 15:02:00,-1.905273,32068.330078,5189.339844,4184.725098,0.945833,2.01,9.637500,86.0,47.875000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
761,2024-03-19 15:31:00,-2.539062,32032.193359,6260.332520,4184.754883,0.929167,2.01,9.937500,86.0,53.674999,NaN
762,2024-03-19 15:32:00,-2.495117,31732.599609,6254.114746,4171.915039,0.875000,2.01,9.987500,86.0,54.349998,NaN
763,2024-03-19 15:33:00,-2.466797,32028.480469,5190.120117,4189.860352,0.945833,2.01,9.987500,86.0,55.549999,NaN
764,2024-03-19 15:34:00,-2.458008,31977.824219,6244.729980,4178.174805,0.929167,2.01,9.987500,86.0,55.099998,NaN
